In [0]:
import numpy as np
import cv2

# Loading the dataset
test_x = np.load("path/to/test_x.npy")    # Kindly add the path to test_x (images)
test_y = np.load("path/to/test_y.npy")    # Kindly add the path to test_y (labels)

print(test_x.shape)
print(test_y.shape)

(464, 512, 512, 3)
(464, 3)


In [0]:
# Normalization
mean_t = np.mean(test_x)
std_t = np.std(test_x)

# Mean and Std value we used for training
# mean_t = 124.64379379055732
# std_t = 62.582209989667376

test_x = test_x - mean_t
test_x = test_x/std_t

In [0]:
# Model 6 Architecture (best performing novel proposition)
import tensorflow as tf
from keras import backend as K
from keras import backend
from keras import models
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, Conv2DTranspose, AveragePooling2D, Activation, add, multiply,Lambda, UpSampling2D, concatenate
import os
from keras import Model
from keras.layers.normalization import BatchNormalization
from keras.applications import imagenet_utils
from keras import layers
from keras.applications.imagenet_utils import decode_predictions


def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same')(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer='glorot_uniform', name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x

def expend_as(tensor, rep,name):
    my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
    return my_repeat

def dense_layer(y, blocks,name):
    for i in range(blocks):
        y = conv_layer(y, 32, name=name + '_block' + str(i + 1))
    return y


def transition_layer(y, reduction, name):
    bn_axis = 3 
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(y)
    y = layers.Activation('relu', name=name + '_relu')(y)
    y = layers.Conv2D(int(backend.int_shape(y)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(y)
    # y = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(y)
    return y


def conv_layer(y, growth_rate, name):
    bn_axis = 3 
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(y)
    y1 = layers.Activation('relu', name=name + '_0_relu')(y1)
    y1 = layers.SeparableConv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(y1)
    y1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(y1)
    y1 = layers.Activation('relu', name=name + '_1_relu')(y1)
    y1 = layers.SeparableConv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(y1)
    y = layers.Concatenate(axis=bn_axis, name=name + '_concatenate')([y, y1])
    return y

def Model6():
    bn_axis = 3

    image=layers.Input((512,512,3))
    y = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(image)
    y = layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(y)
    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1/bn')(y)
    y = layers.Activation('relu', name='conv1/relu')(y)
    y = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(y)
    y = layers.MaxPooling2D(3, strides=2, name='pool1')(y)

    d1 = dense_layer(y, 6, name='conv2')
    g1 = UnetGatingSignal(d1, is_batchnorm=True, name='g1')
    g1 = AveragePooling2D(2, strides=2, name='p1')(g1)
    t1 = transition_layer(d1, 0.5, name='t1')
    attn1 = AttnGatingBlock(t1, g1, 128, '_1')
    attn1 = AveragePooling2D(2, strides=2, name='p3')(attn1)


    d2 = dense_layer(attn1, 12, name='conv3')
    g2 = UnetGatingSignal(d2, is_batchnorm=True, name='g2')
    g2 = AveragePooling2D(2, strides=2, name='p2')(g2)
    t2 = transition_layer(d2, 0.5, name='t2')
    attn2 = AttnGatingBlock(t2, g2, 256, '_2')
    attn2 = AveragePooling2D(2, strides=2, name='p6')(attn2)


    d3 = dense_layer(attn2, 24, name='conv4')
    g3 = UnetGatingSignal(d3, is_batchnorm=True, name='g3')
    g3 = AveragePooling2D(2, strides=2, name='p7')(g3)
    t3 = transition_layer(d3, 0.5, name='t3')
    attn3 = AttnGatingBlock(t3, g3, 512, '_3')
    attn3 = AveragePooling2D(2, strides=2, name='p9')(attn3)



    d4 = dense_layer(attn3, 16, name='conv5')

    y = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(d4)
    y = layers.Activation('relu', name='relu')(y)
    y2 = GlobalAveragePooling2D()(y)
    y3 = Dropout(0.3)(y2)
    output1 = Dense(10,activation='relu')(y3)
    output2 = Dense(3,activation='softmax')(output1)

    model = models.Model(image, output2, name='model_1')
    return model

from keras.optimizers import SGD, Adam, Adadelta
model=Model6()
model.compile(optimizer=Adam(lr=3e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [0]:
# Loading the saved weights
model.load_weights('weights/model6_best.h5')  # Kindly add the path to the weights file here
# The weight file associated with this model is model6_best.h5


In [0]:
# Prediction on the test set 
test_pred = model.predict(test_x, batch_size=8)

ftest = np.zeros(len(test_pred))
atest = np.zeros(len(test_x))


In [0]:
for i in range(len(test_y)):
  if test_y[i][0] >= test_y[i][1] and test_y[i][0] >= test_y[i][2]:
    atest[i] = 0
  elif test_y[i][1] >= test_y[i][0] and test_y[i][1] >= test_y[i][2]:
    atest[i] = 1
  else:
    atest[i] = 2


In [0]:
for i in range(len(test_pred)):
  if test_pred[i][0] >= test_pred[i][1] and test_pred[i][0] >= test_pred[i][2]:
    ftest[i] = 0
  elif test_pred[i][1] >= test_pred[i][0] and test_pred[i][1] >= test_pred[i][2]:
    ftest[i] = 1
  else:
    ftest[i] = 2

In [0]:
# Results section
from sklearn import metrics
print(metrics.confusion_matrix(atest, ftest))

[[ 27   0   0]
 [  0 214   1]
 [  1   6 215]]


In [0]:
print(metrics.classification_report(atest, ftest, digits=3))

              precision    recall  f1-score   support

         0.0      0.964     1.000     0.982        27
         1.0      0.973     0.995     0.984       215
         2.0      0.995     0.968     0.982       222

    accuracy                          0.983       464
   macro avg      0.977     0.988     0.982       464
weighted avg      0.983     0.983     0.983       464

